In [2]:
!git clone https://github.com/MohamedKhalifa1/Stack-Overflow-Annual-Developer-Survey-Analysis

fatal: destination path 'Stack-Overflow-Annual-Developer-Survey-Analysis' already exists and is not an empty directory.


In [33]:
EXPERIMENT_NAME = 'mlflow_jobline'
EXPORT_MODEL_PATH = "mlflow.pkl"
DATA_PATH = '/content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/preprocessed/04_cleaned_data.pkl.gz'
RANDOM_STATE = 0

In [5]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [29]:
import mlflow
from mlflow.tracking import MlflowClient
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import pickle
import gzip

In [13]:
with gzip.open(DATA_PATH, 'rb') as f_in:
    df = pickle.load(f_in)

In [15]:
df.head()

DevType                                 ... skills_clusters                              
          Academic researcher Blockchain  ...  skills_group_7 skills_group_8 skills_group_9
2       0                   0          0  ...               0              0              0
3       0                   0          0  ...               0              0              1
4       0                   0          0  ...               0              0              0
5       0                   0          0  ...               0              0              0
8       0                   0          0  ...               0              0              0

[5 rows x 318 columns]

In [20]:
X = df.drop('DevType', axis=1)
X = X.droplevel(level=0, axis=1)
y = df['DevType']

<ipython-input-20-e42c3163804a>:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = df.drop('DevType', axis=1)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=RANDOM_STATE)

In [26]:
PCA_VAR = 0.85

In [30]:
classifer = make_pipeline(StandardScaler(),
                          PCA(n_components=PCA_VAR, random_state=RANDOM_STATE),
                          RandomForestClassifier())
classifer.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=0.85, random_state=0)),
                ('randomforestclassifier', RandomForestClassifier())])

In [31]:
test_prediction  = classifer.predict(X_test)
accuracy_score = accuracy_score(y_test, test_prediction)
print(f'Accuracy: {accuracy_score}')

Accuracy: 0.16489730594825286


In [32]:
# Export model
with open(EXPORT_MODEL_PATH, 'wb') as handle:
    pickle.dump(classifer, handle)

In [34]:
client = MlflowClient()
mlflow.set_experiment(EXPERIMENT_NAME)
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

2024/06/21 19:30:47 INFO mlflow.tracking.fluent: Experiment with name 'mlflow_jobline' does not exist. Creating a new experiment.


In [35]:
with mlflow.start_run(experiment_id=exp.experiment_id):
    mlflow.log_param("pca_var", PCA_VAR)
    mlflow.log_metric("Accuracy", accuracy_score)
    mlflow.log_artifact(EXPORT_MODEL_PATH)